In [1]:
%cd ..
%load_ext autoreload
%autoreload 2

REMOTE_RTS_DATA = "/media/sinergia/RTS/"
REMOTE_VIDEOS = '/mnt/rts/'

LOCAL_RTS_DATA = "/media/data/rts/"
METADATA = LOCAL_RTS_DATA + 'metadata'
LOCAL_VIDEOS = LOCAL_RTS_DATA + 'archive'

OUTDIR = 'data'


/home/kirell/work/rts


In [2]:
from datetime import datetime
import orjson
import pandas as pd
import zipfile
import os
import io
from typing import Dict, List, Optional, Tuple
from pathlib import Path

In [3]:
# LOCAL imports
import rts
import rts.pipeline
import rts.utils
import rts.io.media
import rts.features.audio
import rts.features.text
import rts.features.image

LOG = rts.utils.get_logger()

/home/kirell/.cache/pypoetry/virtualenvs/rts-506MfBOj-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
rts.features.image.embed_images([Path(OUTDIR) / 'testori1.jpg', 
                                 Path(OUTDIR) / 'testori2.jpg'])

INFO:RTS:Load model: resnet50


array([[0.06273046, 1.1778458 , 0.3374148 , ..., 0.23949899, 0.16084106,
        0.41598725],
       [0.28124833, 0.5253096 , 0.09955239, ..., 0.00275622, 0.01974793,
        0.05470418]], dtype=float32)

In [18]:
video_path = os.path.join(LOCAL_VIDEOS, '0', '0', '0', 'ZB001000', 'ZB001000.mp4')

outpath = Path(OUTDIR) / 'test_trim.mp4'
outpath = rts.io.media.trim(video_path,  Path(OUTDIR) / 'test_trim.mp4', 300, 310)


Args: ['-hide_banner', '-loglevel', 'error', '-i', '/media/data/rts/archive/0/0/0/ZB001000/ZB001000.mp4', '-filter_complex', '[0:v]trim=end=310:start=300[s0];[s0]setpts=PTS-STARTPTS[s1];[0:a]atrim=end=310:start=300[s2];[s2]asetpts=PTS-STARTPTS[s3];[s1][s3]concat=a=1:n=1:v=1[s4][s5]', '-map', '[s4]', '-map', '[s5]', '-movflags', 'faststart', 'data/test_trim.mp4', '-y']


In [4]:
# vidx = rts.metadata.read_video_folder_index(os.path.join(OUTDIR, 'vidx.json'))
# df = rts.metadata.read_all_metadata(METADATA, vidx)
# rts.metadata.metadata_to_hdf5(OUTDIR, 'rts_metadata', df)

df = rts.metadata.load_metadata_hdf5(METADATA, 'rts_metadata')
sample_df = rts.metadata.get_one_percent_sample(df)
# # rts.metadata.export_metadata_stats(sample_df, Path(OUTDIR) / 'sample')
# # rts.metadata.export_metadata_stats(df, OUTDIR)
sample_df.head()

# medias = rts.metadata.load_all_media_info(LOCAL_VIDEOS)
# clips = rts.metadata.load_all_clips(LOCAL_VIDEOS)

# clip_df = rts.metadata.build_clips_df(LOCAL_VIDEOS, METADATA)
# rts.utils.dataframe_to_hdf5(METADATA, 'clips', clip_df)

,guid,mediaFolderPath,mediaDuration,ratio,formatResolution,publishedDate,categoryName,assetType,contentType,backgoundType,collection,publishedBy,rights,title,resume,geoTheme,resumeSequence,created,published,sequences
mediaId,,,,,,,,,,,,,,,,,,,,
ZE002100,AA1101000770,/mnt/rts/0/0/1/ZE002100,1524,16:9,SD,2010-04-29T17:40:00Z,Programme,Programme,Emissions de conseil,None,Dolce vita,TSR 1,Restriction/Condition,CAMPING - 10.04.29,"* Plateau : 20100429, SIKI Muriel reçoit ses i...","[LIEU INDÉTERMINÉ, VAUD, PLATEAU]",[00:01.24\r\nPlateau : diverses présentations ...,1670505072,1272562800,"{'890511': {'guid': 'AA1101000770', 'mediaId':..."
ZS009400,AA1106004840,/mnt/rts/0/0/4/ZS009400,530,16:9,SD,2011-03-05T00:00:00Z,Programme,Programme,Autres émissions religieuses,None,Faut pas croire,TSR 1,Restriction/Condition,11.03.05,"* Plateau : 20110305, BOISSET Christophe reçoi...","[LIEU INDÉTERMINÉ, NEUVEVILLE, LA, PLATEAU]",[00:01.18\r\nPlateau : diverses présentations ...,1670505072,1299283200,"{'897154': {'guid': 'AA1106004840', 'mediaId':..."
ZM001400,AA1201021870,/mnt/rts/0/0/4/ZM001400,43,16:9,SD,2012-01-22T00:00:00Z,Programme,Programme,"Bandes-annonces, lancements",None,Bande Annonce,TSR 2,Inconnu,Alain Delon. 2BLS CINEMA 2201,None,None,None,1670505073,1327190400,{}
ZT002200,AA1202023530,/mnt/rts/0/0/2/ZT002200,4445,16:9,SD,2012-04-08T00:00:00Z,Programme,Retransmission sportive,Retransmissions en direct,None,Motocyclisme,TSR 2,Restriction/Condition,MOTO3 - GRAND PRIX DU QATAR LOSAIL 2012,None,None,None,1670505073,1333843200,{}
ZM002200,AA1203026020,/mnt/rts/0/0/2/ZM002200,60,16:9,HD,2012-04-07T00:00:00Z,Programme,Programme,"Bandes-annonces, lancements",None,Bande Annonce,None,Inconnu,BLS SAMEDI 0704,None,None,None,1670505073,1333756800,{}


In [ ]:
client = rts.db.fulltext.get_client()
midx = rts.db.fulltext.get_or_create_media_index(client)
# Fulltext Index from HDF5
# midx.delete_all_documents()
rts.db.fulltext.index_media(df, midx)
# Wait 30 sec for full indexation
midx.get_stats().number_of_documents  # 522103

In [ ]:
medias = rts.metadata.load_all_media_info(LOCAL_VIDEOS)

In [ ]:
clips = rts.metadata.load_all_clips(LOCAL_VIDEOS)

In [ ]:
df = rts.metadata.build_clips_df(LOCAL_VIDEOS)

In [ ]:
atlases = rts.metadata.create_clip_texture_atlases(df, LOCAL_RTS_DATA,
                                                   '0', # folder name
                                                   tile_size=64,
                                                   flip=True,
                                                   no_border=True, 
                                                   format='jpg')

In [ ]:
import PIL
import os
from PIL import ImageOps, Image

In [ ]:
test_im_path = os.path.join(OUTDIR, 'test_image.jpg')
im = Image.open(test_im_path)
# rts.io.media.save_image_pyramid(im, Path(OUTDIR) / 'test_image_pyramid', 'test_image', True)

In [ ]:
video_path = os.path.join(LOCAL_VIDEOS, '0', '0', '0', 'ZB001000', 'ZB001000.mp4')
# audio_path = os.path.join(LOCAL_VIDEOS, '0', '0', '0', 'ZB001000', 'ZB001000_audio.aac')
# rts.io.media.get_media_info(video_path)

# scenes = rts.io.media.detect_scenes(video_path)
# scenes = rts.io.media.filter_scenes(scenes, 5)
# clips = rts.io.media.save_scenes_images(scenes, video_path, OUTDIR)
# rts.pipeline.save_scenes(clips, OUTDIR)

In [5]:
test_video_folder = os.path.join(LOCAL_VIDEOS, '0', '0', '0', 'ZB001000')
audio_path = os.path.join(test_video_folder, 'ZB001000_audio.m4a')

# outpath = rts.io.media.trim_media(video_path, 30, 40, Path(OUTDIR) / 'test_trim.mp4')
# rts.io.media.get_media_info(outpath)

In [1]:
from whisper_jax import FlaxWhisperPipline
import jax.numpy as jnp

# instantiate pipeline with bfloat16 and enable batching
pipeline = FlaxWhisperPipline("openai/whisper-large-v2", dtype=jnp.bfloat16, batch_size=16)

# transcribe and return timestamps
outputs = pipeline(audio_path, task="transcribe", return_timestamps=True)

/home/kirell/.cache/pypoetry/virtualenvs/rts-506MfBOj-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-05-11 12:06:49.241419: E external/xla/xla/stream_executor/cuda/cuda_dnn.cc:427] Loaded runtime CuDNN library: 8.7.0 but source was compiled with: 8.8.0.  CuDNN library needs to have matching major version and equal or higher minor version. If using a binary install, upgrade your CuDNN library.  If building from sources, make sure the library loaded at runtime is compatible with the version specified during compile configuration.


XlaRuntimeError: FAILED_PRECONDITION: DNN library initialization failed. Look at the errors above for more details.

In [9]:
%timeit
# transcript = rts.pipeline.transcribe_media(
#     test_video_folder,
#     audio_path,
#     force=True
# )
# d = rts.features.audio.transcribe_media(audio_path, 'large-v2')
d
# t = rts.features.text.find_locations(transcript)

[{'t': '...', 's': '30.00', 'e': '42.40'},
 {'t': '...', 's': '42.60', 'e': '48.40'},
 {'t': 'Les employés romands de Suisse Air sont passés aux actes.',
  's': '48.60',
  'e': '50.88'},
 {'t': 'Ils ont bloqué plusieurs vols à coups intrins.',
  's': '51.08',
  'e': '53.32'},
 {'t': 'Ils refusent de voir leur salaire amputé.',
  's': '53.52',
  'e': '55.88'},
 {'t': '...', 's': '56.08', 'e': '57.24'},
 {'t': 'Le chauffeur de camion qui a provoqué la tragédie du Gotthard',
  's': '57.44',
  'e': '59.72'},
 {'t': "était sous l'emprise de l'alcool.", 's': '59.72', 'e': '61.36'},
 {'t': "C'est une des révélations de l'enquête.", 's': '61.56', 'e': '63.60'},
 {'t': '...', 's': '63.80', 'e': '65.60'},
 {'t': 'Et puis, les femmes sont descendues dans la rue à Kaboul.',
  's': '65.80',
  'e': '67.84'},
 {'t': 'Elles demandent plus de droits et plus de libertés.',
  's': '68.04',
  'e': '70.68'},
 {'t': '...', 's': '70.88', 'e': '74.08'},
 {'t': '...', 's': '74.28', 'e': '87.28'},
 {'t': '...',

In [ ]:
clips = rts.pipeline.extract_location_clips_from_transcript(transcript,
    25,
    OUTDIR,
    video_path,
    min_seconds=3,
    extend_duration=6,
    num_images=3,
    force=True
)

In [ ]:
imgs = list(rts.utils.recursive_glob(Path(OUTDIR) / 'images' / 'original', '.jpg'))
imgs

In [ ]:
rts.io.media.create_square_atlases(
    imgs,
    Path(OUTDIR) / 'atlas',
    width=512,
    max_tile_size=128,
    no_border=True,
    flip=True,
    atlas_prefix='small_atlas',
    bg_color=(255, 255, 255, 255)
)

In [ ]:
res = rts.features.text.enrich_scenes_with_transcript(clips.copy(), t)
# res

In [ ]:
swiss_cities = rts.features.text.get_swiss_cities()

In [ ]:
input_video_folder = os.path.join(REMOTE_VIDEOS, '0', '0', '0', 'ZB001000')
# input_video_folder = os.path.join(VIDEOS, '0', '0', 'ZB002000')
# v, a = rts.io.archive.get_raw_video_audio_parts(input_video_folder)

# '/'.join(input_video_folder.split('/')[-4:])
# remuxed = rts.pipeline.create_optimized_media(input_video_folder, LOCAL_VIDEOS, force=True)
# print(remuxed)

rts.pipeline.process_media(input_video_folder, LOCAL_VIDEOS, 
    compute_transcript=True, force_scene=False, force_trans=False)

In [ ]:
%ls {remux_folder}

In [ ]:
fts.groupby('mediaId').count().sort_values('location', ascending=False)

In [ ]:
fts

In [ ]:
import weaviate

client = weaviate.Client("http://localhost:8080") # or another location where your Weaviate instance is running

client.schema.get() # get the full schema as example

In [ ]:
swiss_cities = rts.features.text.get_swiss_cities()

In [ ]:
swiss_cities['Genève']

In [ ]:
rts.utils.obj_to_json(r, os.path.join(OUTDIR, 'transcript.json'))

In [ ]:
rts.io.media.to_mp3(a, os.path.join(OUTDIR, 'audio.mp3'))

In [ ]:
rts.io.media.to_mp3(a, os.path.join(OUTDIR, 'audio.mp3'))

In [ ]:
rts.io.media.extract_audio(a, os.path.join(OUTDIR, 'myaudio'))

In [ ]:
import av
import av.datasets


video_part = os.path.join(VIDEOS, '0', '0', 'ZB001000')
# video_part = os.path.join(VIDEOS, '0', '0', 'ZB002000')
v, a = rts.io.archive.get_raw_video_audio_parts(video_part)
out_path = os.path.join(OUTDIR, 'thumbs')

content = v

print(content)

with av.open(content) as container:
    # Signal that we only want to look at keyframes.
    stream = container.streams.video[0]
    stream.codec_context.skip_frame = "NONKEY"

    # print(stream.codec_context.height)
    for i, frame in enumerate(container.decode(stream)):
        if i % 4 != 0:
            continue


    #     print(frame)
    #     print(frame.time, frame.time_base)
        # We use `frame.pts` as `frame.index` won't make must sense with the `skip_frame`.
        frame.to_image().save(
            os.path.join(out_path, f"f{frame.pts:04d}.jpg"),
            quality=80,
        )

In [ ]:
thumbs_dir = os.path.join(OUTDIR, 'thumbs')
outfast = './notebook/fastdup'
fastdup_data = './notebook/out'

In [ ]:
import fastdup

In [ ]:

fastdup.run(input_dir=thumbs_dir, work_dir=fastdup_data, nearest_neighbors_k=8, turi_param='ccthreshold=0.90')    #main running function.
# fastdup.create_duplicates_gallery('out/similarity.csv', save_path=outfast)     #create a visual gallery of found duplicates
# fastdup.create_outliers_gallery('out/outliers.csv',   save_path=outfast)       #create a visual gallery of anomalies
# fastdup.create_components_gallery(fastdup_data, save_path=outfast)                    #create visualiaiton of connected components
# fastdup.create_stats_gallery('out', save_path=outfast, metric='blur')          #create visualization of images stastics (for example blur)
# fastdup.create_similarity_gallery('out', save_path=outfast,get_label_func=lambda x: x.split('/')[-2])     #create visualization of top_k similar images assuming data have labels which are in the folder name
# fastdup.create_aspect_ratio_gallery('out', save_path=outfast)

In [ ]:
config = fastdup.load_config(fastdup_data)
df = fastdup.galleries.do_find_top_components(work_dir=fastdup_data, input_dir=config['input_dir'], kwargs={})